In [1]:
#reading the csv file
import pandas as pd
data = pd.read_csv(r'C:\Users\Abdul Haseeb\Desktop\Recommender system\Recommender-System\Amazon_Beauty_Recommendation.csv')
print(data.head())

           UserId   ProductId       ProductType  Rating   Timestamp  \
0  A3NHUQ33CFH3VM  B00LLPT4HI  Eyeliner & Kajal       5  1405814400   
1  A1TIRNQ7O4REOH  B00LLPT4HI  Eyeliner & Kajal       4  1405987200   
2  A2Y36BR4YSY9F7  B00LLPT4HI  Eyeliner & Kajal       5  1405728000   
3  A23H6FAOLEMAKC  B00LLPT4HI  Eyeliner & Kajal       5  1405814400   
4  A3CHYZGF3OO6WD  B00LLPT4HI  Eyeliner & Kajal       5  1405641600   

                                                 URL  
0  https://www.amazon.in/Maybelline-Colossal-Kaja...  
1  https://www.amazon.in/Maybelline-Colossal-Kaja...  
2  https://www.amazon.in/Maybelline-Colossal-Kaja...  
3  https://www.amazon.in/Maybelline-Colossal-Kaja...  
4  https://www.amazon.in/Maybelline-Colossal-Kaja...  


In [7]:
# Extract product features for content-based filtering
product_features = data[['ProductId', 'ProductType']].drop_duplicates()

# Encode ProductType as numerical features
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
product_features['ProductTypeEncoded'] = encoder.fit_transform(product_features['ProductType'])

print("Product Features Prepared:")
print(product_features.head())


Product Features Prepared:
     ProductId       ProductType  ProductTypeEncoded
0   B00LLPT4HI  Eyeliner & Kajal                   7
27  B00LCEROA2  Eyeliner & Kajal                   7
50  B00LAOMUUS  Eyeliner & Kajal                   7
66  B00L6Q3BH6  Eyeliner & Kajal                   7
85  B00L5JHZJO  Eyeliner & Kajal                   7


In [9]:
from scipy.sparse import coo_matrix

# Extract required columns
rows = data['UserId'].astype('category').cat.codes
cols = data['ProductId'].astype('category').cat.codes
ratings = data['Rating'].values

# Create a sparse matrix
interaction_matrix_sparse = coo_matrix((ratings, (rows, cols)))

print("Sparse User-Item Interaction Matrix Shape:", interaction_matrix_sparse.shape)

# Create mappings for UserId and ProductId
user_mapping = dict(enumerate(data['UserId'].astype('category').cat.categories))
product_mapping = dict(enumerate(data['ProductId'].astype('category').cat.categories))

print("Number of Users:", len(user_mapping))
print("Number of Products:", len(product_mapping))


Sparse User-Item Interaction Matrix Shape: (883753, 23838)
Number of Users: 883753
Number of Products: 23838


In [10]:
from scipy.sparse.linalg import svds
import numpy as np

# Decompose the sparse matrix using SVD
interaction_matrix = interaction_matrix_sparse.toarray()
user_ratings_mean = np.mean(interaction_matrix, axis=1)
interaction_matrix_normalized = interaction_matrix - user_ratings_mean.reshape(-1, 1)

# Compute SVD
U, sigma, Vt = svds(interaction_matrix_normalized, k=50)  # Adjust k as needed
sigma = np.diag(sigma)

# Reconstruct the ratings matrix
predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
print("Predicted Ratings Matrix Shape:", predicted_ratings.shape)

# Function to recommend products
def recommend_products_cf(user_id, user_mapping, product_mapping, num_recommendations=10):
    user_idx = list(user_mapping.keys())[list(user_mapping.values()).index(user_id)]
    sorted_ratings = np.argsort(predicted_ratings[user_idx])[::-1]
    
    recommendations = [
        {
            "ProductId": product_mapping[prod_idx],
            "Predicted_Rating": predicted_ratings[user_idx, prod_idx],
        }
        for prod_idx in sorted_ratings[:num_recommendations]
    ]
    return recommendations

# Example recommendation
user_id = list(user_mapping.values())[0]
cf_recommendations = recommend_products_cf(user_id, user_mapping, product_mapping)
print(f"Collaborative Filtering Recommendations for User {user_id}:")
for rec in cf_recommendations:
    print(rec)


MemoryError: Unable to allocate 157. GiB for an array with shape (883753, 23838) and data type int64